# Code testing for "YOLOv3" step method

In [1]:
import tensorflow as tf
from module.DirectoryProcessor import DirectoryProcessor
from module.object_detection.YOLOv3DataGenerator import YOLOv3DataGenerator
from module.object_detection.YOLOv3Visualizer import YOLOv3Visualizer

In [2]:
CFG = {
    "dataset_dir":  "D:\Github Public\DATASET MANAGER\ObjectDetection_SIMPLE",
    "image_size": 320,
    "anchor": 3,
    "batch": 4,
    "label": ["Face"],
    "optimizer": tf.keras.optimizers.Adam(learning_rate=1e-4),
}

In [3]:
path_list = DirectoryProcessor.get_all_files(CFG["dataset_dir"])
path_list = [path for path in path_list if "_ROI" not in path]
path_list = [path for path in path_list if ".csv" not in path]
path_list = [(path_list[i], path_list[i+1]) for i in range (0,len(path_list),2)]

## Load image with no augmentation to datagenerator

In [4]:
data_generator = YOLOv3DataGenerator(
    input=path_list,
    label_list=CFG["label"],
    image_size=CFG["image_size"], 
    num_anchor=CFG["anchor"], 
    horizontal_flip=False, 
    vertical_flip=False, 
    translate_range=None, 
    rotation_range=None,
)

dataset = data_generator.generate_dataset(
    batch_size=CFG["batch"],
    drop_reminder=False,
)

#visualizer = YOLOv3Visualizer(CFG["label"])
#visualizer.display_batch(dataset, figsize=(13, 16), show_grid=True, show_highlight=True)

Calculating anchor size ... 3 anchor boxes: [(0.54, 0.54), (0.45, 0.45), (0.49, 0.49)]
CALLING preprocessing function
READ IMAGE ... COMPLETE
READ XML ... COMPLETE
AUGMENTATION: padding -> COMPLETE
Generate annotation labels ... COMPLETE


## Model

In [5]:
from model.YOLOv3 import YOLOv3

yolo = YOLOv3(image_size=CFG["image_size"], num_anchor=CFG["anchor"], num_class=len(CFG["label"]))
#yolo.summary()

## Loss and metric

In [6]:
from loss.YOLOLoss import YOLOLoss
from metric.MeanAveragePrecision import MeanAveragePrecision

loss = YOLOLoss()
mAP = MeanAveragePrecision(num_class=len(CFG["label"]))

yolo.compile(optimizer=CFG["optimizer"], loss=loss, metrics=[mAP])

In [7]:
dataset

<PrefetchDataset shapes: ((None, 320, 320, 3), ((None, 10, 10, 3, 6), (None, 20, 20, 3, 6), (None, 40, 40, 3, 6))), types: (tf.float32, (tf.float32, tf.float32, tf.float32))>

In [9]:
yolo.train(dataset, epochs=100)

Continuing training from epoch 10
Epoch 11/100
10/10 [==============================] - 12s 1s/step - loss: 8.9459 - small_loss: 4.5841 - medium_loss: 2.5032 - large_loss: 1.8586 - small_mAP: 0.0000e+00 - medium_mAP: 0.0000e+00 - large_mAP: 0.0000e+00
Epoch 12/100
10/10 [==============================] - 11s 1s/step - loss: 6.2135 - small_loss: 2.7605 - medium_loss: 1.8245 - large_loss: 1.6285 - small_mAP: 0.0000e+00 - medium_mAP: 0.0000e+00 - large_mAP: 0.0000e+00
Epoch 13/100
10/10 [==============================] - 11s 992ms/step - loss: 6.0523 - small_loss: 2.1785 - medium_loss: 2.2248 - large_loss: 1.6490 - small_mAP: 0.0000e+00 - medium_mAP: 0.0000e+00 - large_mAP: 0.0000e+00
Epoch 14/100
10/10 [==============================] - 11s 997ms/step - loss: 5.1644 - small_loss: 2.0408 - medium_loss: 1.6952 - large_loss: 1.4283 - small_mAP: 0.0000e+00 - medium_mAP: 0.0000e+00 - large_mAP: 0.0000e+00
Epoch 15/100
10/10 [==============================] - 11s 998ms/step - loss: 5.5022 - sm

KeyboardInterrupt: 